In [1]:
from os import device_encoding

import torch
from torch import nn
from torch.nn import functional as F
import math

In [2]:
# 伪代码
class MultiHeadAttention(nn.Module):
    def __init__(self,*args):
        super().__init__()

In [3]:
# 伪代码
class LayerNorm(nn.Module):
    def __init__(self,*args):
        super().__init__()

In [7]:
# 伪代码
class PositionWiseFeedForward(nn.Module):
    def __init__(self,*args):
        super().__init__()

In [ ]:
# 伪代码
class TransformerEmbedding(nn.Module):
    def __init__(self,*args):
        super().__init__()

* 输入向量：$\mathbf{x}$
* Encoder 输出（记作 memory）：$\mathbf{M}$
* 各个子层的结果：依次是 $\mathbf{x}_1, \mathbf{x}_2, \mathbf{y}$

## DecoderLayer 的公式

### 1. Masked Multi-Head Self-Attention

对目标序列的输入 $\mathbf{x}$，做带掩码的多头注意力：

$$
\mathbf{z}_1 = \text{MultiHeadAttention}(\mathbf{x}, \mathbf{x}, \mathbf{x}, \text{mask})
$$

加上残差 & LayerNorm：

$$
\mathbf{x}_1 = \text{LayerNorm}(\mathbf{x} + \mathbf{z}_1)
$$

---

### 2. Encoder–Decoder Attention (Cross-Attention)

以 $\mathbf{x}_1$ 作为 Query，Encoder 的输出 $\mathbf{M}$ 作为 Key 和 Value：

$$
\mathbf{z}_2 = \text{MultiHeadAttention}(\mathbf{x}_1, \mathbf{M}, \mathbf{M})
$$

加上残差 & LayerNorm：

$$
\mathbf{x}_2 = \text{LayerNorm}(\mathbf{x}_1 + \mathbf{z}_2)
$$

---

### 3. Position-wise Feed Forward Network (FFN)

FFN 通常是两层前馈网络：

$$
\text{FFN}(\mathbf{x}_2) = \max(0, \mathbf{x}_2 \mathbf{W}_1 + \mathbf{b}_1)\mathbf{W}_2 + \mathbf{b}_2
$$

加上残差 & LayerNorm：

$$
\mathbf{y} = \text{LayerNorm}(\mathbf{x}_2 + \text{FFN}(\mathbf{x}_2))
$$

---

## Multi-Head Attention 的细节公式

每个 **注意力头（head）**：

$$
\text{Attention}(Q, K, V) = \text{softmax}\!\left(\frac{QK^\top}{\sqrt{d_k}} + \text{mask}\right)V
$$

多头拼接后：

$$
\text{MultiHead}(Q,K,V) = \text{Concat}(\text{head}_1, \dots, \text{head}_h)\mathbf{W}^O
$$

其中：

$$
\text{head}_i = \text{Attention}(Q\mathbf{W}^Q_i, K\mathbf{W}^K_i, V\mathbf{W}^V_i)
$$

In [5]:
class DecoderLayer(nn.Module):
    def __init__(self,d_model,ffn_hidden,drop_prod,n_head):
        super().__init__()
        self.d_model=d_model
        self.ffn_hidden=ffn_hidden
        self.drop_prod=drop_prod
        self.n_head=n_head
        # Masked Self-Attention
        self.self_attention=MultiHeadAttention(self.d_model,self.n_head)
        # Cross Attention
        self.cross_attention=MultiHeadAttention(self.d_model,self.n_head)
        # Position-Wise-Feed-Forward
        self.ffn=PositionWiseFeedForward(self.d_model,self.ffn_hidden,self.drop_prod)
        # LayerNorm
        self.norm1=LayerNorm(self.d_model)
        self.norm2=LayerNorm(self.d_model)
        self.norm3=LayerNorm(self.d_model)
        # Dropout
        self.dropout1=nn.Dropout(self.drop_prod)
        self.dropout2=nn.Dropout(self.drop_prod)
        self.dropout3=nn.Dropout(self.drop_prod)
    def forward(self,dec,enc,t_mask,s_mask):
        x1=self.norm1(self.dropout1(self.self_attention(dec,dec,dec,t_mask))+dec)
        x2=self.norm2(self.dropout2(self.cross_attention(x1,enc,enc,s_mask)+x1))
        y=self.norm3(x2+self.dropout3(self.ffn(x2)))
        return y

好的！以下是 Transformer Decoder 部分的详细计算公式。我们将分步骤解析每个组件的数学表达。

---

## Decoder 整体计算流程

设：
- `X`：目标序列的词嵌入（shifted right） + 位置编码，形状为 `(batch_size, tgt_len, d_model)`
- `Z`：Encoder 的输出，形状为 `(batch_size, src_len, d_model)`
- `M_t`：目标序列的掩码矩阵（防止看到未来信息）

---

## 1. Masked Multi-Head Self-Attention

这是 Decoder 的第一子层，确保自回归性质。

### 计算公式：

**a) 线性变换得到 Q, K, V：**
$$
\begin{align*}
Q &= X \cdot W_Q \\
K &= X \cdot W_K \\
V &= X \cdot W_V
\end{align*}
$$

**b) Scaled Dot-Product Attention（带掩码）：**
$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}} + M_t\right)V
$$

**c) Multi-Head 拼接：**
$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W_O
$$
其中 $\text{head}_i = \text{Attention}(XW_Q^i, XW_K^i, XW_V^i)$

**d) Add & Norm：**
$$
X_1 = \text{LayerNorm}(X + \text{Dropout}(\text{MultiHead}(X, X, X)))
$$

---

## 2. Multi-Head Cross-Attention

这是连接 Encoder 和 Decoder 的关键桥梁。

### 计算公式：

**a) 线性变换（Q来自Decoder，K,V来自Encoder）：**
$$
\begin{align*}
Q &= X_1 \cdot W_Q \\
K &= Z \cdot W_K \\
V &= Z \cdot W_V
\end{align*}
$$

**b) Scaled Dot-Product Attention：**
$$
\text{CrossAttention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

**c) Multi-Head 拼接：**
$$
\text{MultiHeadCross}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W_O
$$

**d) Add & Norm：**
$$
X_2 = \text{LayerNorm}(X_1 + \text{Dropout}(\text{MultiHeadCross}(X_1, Z, Z)))
$$

---

## 3. Position-wise Feed-Forward Network

与前两个注意力子层不同，FFN 独立处理每个位置。

### 计算公式：

**a) 两层线性变换 + 激活函数：**
$$
\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2
$$

**b) Add & Norm：**
$$
X_3 = \text{LayerNorm}(X_2 + \text{Dropout}(\text{FFN}(X_2)))
$$

---

## 4. 输出层（线性变换 + Softmax）

Decoder 堆叠 N 层后的最终输出处理：

### 计算公式：

**a) 线性变换到词汇表大小：**
$$
\text{Logits} = X_3 W_{\text{vocab}} + b_{\text{vocab}}
$$
其中 $W_{\text{vocab}} \in \mathbb{R}^{d_{\text{model}} \times |V|}$

**b) Softmax 得到概率分布：**
$$
P(y_t | y_{<t}, X) = \text{softmax}(\text{Logits})
$$

---

## 完整公式总结

对于第 `l` 个 Decoder Layer：

$$
\begin{align*}
X^{(l)}_1 &= \text{LayerNorm}^{(l)}_1\left(X^{(l-1)} + \text{Dropout}\left(\text{MaskedSelfAttention}^{(l)}(X^{(l-1)})\right)\right) \\
X^{(l)}_2 &= \text{LayerNorm}^{(l)}_2\left(X^{(l)}_1 + \text{Dropout}\left(\text{CrossAttention}^{(l)}(X^{(l)}_1, Z)\right)\right) \\
X^{(l)} &= \text{LayerNorm}^{(l)}_3\left(X^{(l)}_2 + \text{Dropout}\left(\text{FFN}^{(l)}(X^{(l)}_2)\right)\right)
\end{align*}
$$

其中 $X^{(0)} = \text{Embedding}(Y) + \text{PE}$（目标序列嵌入+位置编码）

---

## 掩码矩阵 $M_t$ 的数学定义

$$
M_t(i, j) =
\begin{cases}
0 & \text{if } i \geq j \text{ (允许关注当前位置及之前的位置)} \\
-\infty & \text{if } i < j \text{ (禁止关注未来的位置)}
\end{cases}
$$

In [6]:
class Decoder(nn.Module):
    def __init__(self,dev_voc_size,max_len,d_model,ffn_hidden,n_head,drop_prod,device,n_layer):
        super().__init__()
        self.dev_voc_size=dev_voc_size
        self.max_len=max_len
        self.d_model=d_model
        self.ffn_hidden=ffn_hidden
        self.n_head=n_head
        self.drop_prod=drop_prod
        self.device=device
        self.n_layer=n_layer
        # Embedding
        self.embedding=TransformerEmbedding(self.dev_voc_size,self.max_len,self.drop_prod,self.device)
        # Layers
        self.layers=nn.ModuleList([
            DecoderLayer(self.d_model,self.ffn_hidden,self.drop_prod,self.n_head)
            for _ in range(self.n_layer)
        ])
        # 将结果映射到词汇表上
        self.fc=nn.Linear(in_features=self.d_model,out_features=self.dev_voc_size)
    def forward(self,dec,enc,t_mask,s_mask):
        dce=self.embedding(enc)
        for layer in self.layers:
            dec=layer(dec,enc,t_mask,s_mask)
        dec=self.fc(dec)
        return dec